## Q3.1 Do people care more about lyrics?

In [208]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scripts.helpers import read_raw_dataset, read_saved_dataframe, save_dataframe, debug_dataframe

In [209]:
# load dataset
df_msd_songs   = read_raw_dataset("msd_songs")
df_msd_summary = read_raw_dataset("msd_summary")
df_mxm_lyrics = read_raw_dataset("mxm_lyrics")
df_mxm_words = read_raw_dataset("mxm_words")

In [210]:
#Here we take the top 2000 songs sorted by hotttnesss < 1 by precaution might be some wrong values ?
worst = False
if(worst):
    df_top_songs = df_msd_summary[['song_hotttnesss', 'track_id',"title","artist_name","duration"]].sort_values('song_hotttnesss',ascending=False).loc[df_msd_summary['song_hotttnesss'] <= .2].head(10000)
else:
    df_top_songs = df_msd_summary[['song_hotttnesss', 'track_id',"title","artist_name","duration"]].sort_values('song_hotttnesss',ascending=False).loc[df_msd_summary['song_hotttnesss'] <= 1].head(2000)
df_top_songs = df_top_songs.set_index('track_id')
df_top_songs.head()

,song_hotttnesss,title,artist_name,duration
track_id,,,,
TRYTLWJ128F1482624,1.0,The Killing Moon,Echo And The Bunnymen,347.34975
TRRVILL128F9307C9F,1.0,Panic Switch (UK edit),Silversun Pickups,253.75302
TRNSQDY128F9366831,1.0,This Is War,30 Seconds To Mars,326.97424
TRBFNSL128F42776F9,1.0,Immigrant Song (Album Version),Led Zeppelin,145.05751
TRQJUSS128E07861A6,1.0,Little Sister,Queens Of The Stone Age,211.56526


In [211]:
df_mxm_lyrics.sort_values('count',ascending=False).head()

,track_id,mxm_tid,word,count,is_test
2175736,TRDIEOU128F4261A22,775220,i,1110,0
2175737,TRDIEOU128F4261A22,775220,the,1036,0
2175786,TRDIEOU128F4261A22,775220,woman,718,0
2175762,TRDIEOU128F4261A22,775220,see,700,0
2175743,TRDIEOU128F4261A22,775220,it,638,0


In [212]:
#We remove all lines containing stopwords
stop_wrds = open("./datasets/AdditionalFiles/stop_words.txt","r")
lines = stop_wrds.read().split('\n')
df_mxm_lyrics = df_mxm_lyrics[~df_mxm_lyrics.word.isin(lines)]

In [213]:
df_mxm_lyrics.sort_values('count',ascending=False).head()

,track_id,mxm_tid,word,count,is_test
2175786,TRDIEOU128F4261A22,775220,woman,718,0
2175778,TRDIEOU128F4261A22,775220,da,450,0
2175773,TRDIEOU128F4261A22,775220,face,319,0
2175771,TRDIEOU128F4261A22,775220,wanna,313,0
16666911,TRZSQUD12903CD0378,5574431,ah,294,0


In [214]:
df_lyrics = df_mxm_lyrics
df_lyrics = df_lyrics.groupby('track_id')['count','word'].max()
df_lyrics.head()

,count,word
track_id,,
TRAAAAV128F421A322,2,worth
TRAAABD128F429CF47,7,yeah
TRAAAED128E0783FAB,11,whi
TRAAAEF128F4273421,17,world
TRAAAEW128F42930C0,11,wo


In [215]:
df_lyrics.sort_values('count',ascending=False).head()

,count,word
track_id,,
TRDIEOU128F4261A22,718,woman
TRZSQUD12903CD0378,294,whisper
TRVNYYP128F149293F,293,zombi
TRRCLWL12903CA3682,288,z
TRPHBAP128F4291635,269,year


In [216]:
df_top_songs_word = pd.merge(df_top_songs, df_lyrics, left_index=True, right_index=True, how='inner')
#get rid wrong durations some song is .7 seconds with 7 words occurences which is I think not physically possible
df_top_songs_word = df_top_songs_word[df_top_songs_word['duration'] > 1]
df_top_songs_word.head()

,song_hotttnesss,title,artist_name,duration,count,word
track_id,,,,,,
TRRVILL128F9307C9F,1.0,Panic Switch (UK edit),Silversun Pickups,253.75302,9,worth
TRNSQDY128F9366831,1.0,This Is War,30 Seconds To Mars,326.97424,22,world
TRENTGL128E0780C8E,1.0,Clocks,Coldplay,307.51302,9,want
TRIRQPO128F4281996,1.0,We Are The Sleepyheads,Belle & Sebastian,213.73342,3,whenev
TRQFXKD128E0780CAE,1.0,The Scientist,Coldplay,311.27465,17,tell


In [217]:
df_top_songs_word["word"].value_counts().head()

yeah     152
world     72
wo        56
year      55
wrong     55
Name: word, dtype: int64

The top word is by far 'Yeah' so as a first conclusion we might say people do not really care about lyrics

todo next : add weight to top words in function of occurences per duration of song

In [218]:
#weighted version
pd.options.display.max_rows = 120
df_top_songs_word["weight"] = df_top_songs_word["count"]/df_top_songs_word["duration"]
df_top_songs_word.groupby('word').sum().sort_values('weight',ascending=False)[["count","weight"]].head()

,count,weight
word,,
yeah,2532,12.550327
world,926,4.128827
year,724,3.157062
yes,608,3.105634
young,642,2.997643


Appart from 'yeah' we can see a lot of top words concerning themes such as youth, the world/people, and verbs that refer to desire(wish, want... (woman ?)) these are similar for the worst songs.

In [219]:
df_mxm_all_wrds = df_mxm_lyrics.groupby('word')['count','word'].sum()
df_mxm_all_wrds.sort_values('count',ascending=False).head()

,count
word,
love,298043
know,273137
like,227624
just,226250
time,181248


Here we have a different and interesting result over all the songs we can see that the most recurent word is 'love' and there are many other high-ranked words that recall feelings (feel, like, want, baby, heart, girl...).

In [226]:
df_top_songs_all_wrds = pd.merge(df_top_songs, df_mxm_lyrics, left_index=True, right_on='track_id', how='inner')
df_top_songs_all_wrds = df_top_songs_all_wrds.groupby('word')['count','word'].sum()
df_top_songs_all_wrds.sort_values('count',ascending=False).head()

,count
word,
know,1909
love,1719
like,1511
oh,1504
just,1433


We have quiet similar results for top/worst songs, in terms of lyrics we can say that top/worst songs are very similar in terms of themes addressed.

## Q3.2 Can we measure the quality and popularity of a song from its lyrics? Do the popular songs have the good quality of lyrics from literacy view points?

### "Slang" words part

In [221]:
import csv
with open('./datasets/AdditionalFiles/bad_words.csv', 'r') as f:
    reader = csv.reader(f)
    bad_words = list(reader)
bad_words = [row[1][:len(row[1])-1] for row in bad_words][4:]

In [222]:
df_mxm_bad = df_mxm_lyrics[df_mxm_lyrics.word.isin(bad_words)].sort_values("count",ascending = False)
df_mxm_bad.head(10)

,track_id,mxm_tid,word,count,is_test
16171494,TRYYFQN12903CB0F33,3055464,fuck,110,0
12011358,TRSLSAA128F930C4E6,3055461,fuck,103,0
3440095,TRFGLAV128F428B3AA,5363431,nigga,96,0
15280866,TRXOWQG128F4235B6D,558948,fuck,93,0
12840660,TRTTHRI128E0799176,1785985,nigga,92,0
9547398,TRORVNJ128F1453BEF,3258525,nigga,92,0
5822612,TRIZNAZ12903CA251C,2977352,nigga,92,0
11081828,TRRATGA128E078189D,7957066,bang,91,0
2718076,TREDUPG128F4291E3E,6835906,kill,91,0
8818492,TRNOTHV128F4291E4C,2069315,kill,91,0


In [223]:
df_bad_count = df_mxm_bad.groupby('track_id')['count','word'].sum()
df_top_bad_songs = pd.merge(df_top_songs, df_bad_count, left_index=True, right_index=True, how='inner')
print("We have a ratio of "+repr(len(df_top_bad_songs)/len(df_top_songs_word))+" top songs with bad words")
df_top_bad_songs.sort_values("count",ascending=False).head()

We have a ratio of 0.3063533507397737 top songs with bad words


,song_hotttnesss,title,artist_name,duration,count
track_id,,,,,
TRQAKCH12903CB2C5A,0.905246,I Got 5 On It,The Luniz,253.38730,57
TRMJNMB128F149431F,0.903538,Gimme The Loot (Album Version),The Notorious B.I.G.,285.93587,42
TRKMBDM128E0783E27,0.880696,Hollaback Girl,Gwen Stefani,199.91465,38
TRUNWLW128F92D2CCD,0.896012,Hush,Kula Shaker,175.09832,36
TRFUXCO128F423B33E,0.924924,Hood Figga (Amended Album Version),Gorilla Zoe,200.93342,35


We have a ratio of 0.3063533507397737 top songs with bad words. For the low hotttnesss songs we can actually see that we only have 0.22113502935420742 as a ratio. So people might be more interested in borderline songs ?

### Lyrics density

In [224]:
df_density = df_mxm_lyrics.groupby("track_id").sum()
df_density = pd.merge(df_top_songs, df_density, left_index=True, right_index=True, how='inner')
df_density['density'] = df_density['count'] /df_density['duration']
df_density = df_density.sort_values('density',ascending = True)
df_density.head(10)

,song_hotttnesss,title,artist_name,duration,mxm_tid,count,is_test,density
track_id,,,,,,,,
TRNCXNX128F4282732,0.927425,Festival,Sigur Rós,565.13261,21852384,3,0,0.005308
TRLQUGG128F4261B0E,0.915077,Olsen Olsen,Sigur Ros,483.13424,4405912,6,0,0.012419
TRNKWUD128F92FEB41,0.900961,Music Is Math,Boards of Canada,321.46240,3361132,4,0,0.012443
TRWUKDU128F92E4963,0.879231,Om Nashi Me,Edward Sharpe & The Magnetic Zeros,376.05832,26090667,5,0,0.013296
TRFAOTA128E07989E0,0.940713,Heysátan,Sigur Rós,249.62567,11900784,6,0,0.024036
TRKEDJH128F1486DED,0.909013,Rhineland (Heartland),Beirut,238.81098,20076508,7,0,0.029312
TRDZVAX128F427C6FA,0.891275,Untrust us,Crystal Castles,186.38322,41753538,7,0,0.037557
TRBHUYM128E07989DB,0.965311,Sæglópur,Sigur Rós,458.50077,43636109,19,0,0.041439
TRIPOMF128F1455251,0.911518,ný Batterý,Sigur Ros,470.15138,8811800,21,0,0.044666


In [225]:
#We do this because theres a lot of wrong word count/duration
df_density= df_density.loc[df_density['density'] < 1.9]
print("On average we have "+repr(df_density['density'].mean())+" words per second for the top songs")

On average we have 0.45214054682294735 words per second for the top songs


We have 0.42832003048718376 for the worst and 0.45214054682294735 for the top the difference is quiet small and I do not think we can come to a conclusion due to the fact there might be many errors in the word counts